# Z alignemnt and PSF for different imaging ('pockels') and stimulation ('uncaging') laser wavelengths
Check the associated protocol `z_align_psf.md` for more details (the most important thing being the data organisation conventions).

The only thing a user needs to change in this script is the `calibration` variable (in the cell just below), to match it to the desired zstacks parent directory (in this case `calibration = 'YYYY-MM-DD_zpsf'` or `calibration = 'YYYY-MM-DD_zpsf'`))

In [ ]:
calibration = '2025-10-10_zpsf'

The part below does not need any changing.

In [ ]:
import numpy as np
import tifffile as tf
import os
import matplotlib.pyplot as plt

import napari 

from scipy.optimize import curve_fit

def gaussian(x, a, x0, sigma):
    return a * np.exp(-(x - x0)**2 / (2 * sigma**2))

# if not in folder name 'photostim_deve', change to it
while not os.getcwd().endswith('photostim_deve'):
    os.chdir('..')  # Go to root of repo

In [ ]:
path = f'data_raw/calibration/{calibration}/'
save_path = 'utils/calibration_z_align_psf/'
z_step = 0.5

In [ ]:
# find all subfolders in path that contain nm
all_zstack_path = [os.path.join(path, f) for f in os.listdir(path) if os.path.isdir(os.path.join(path, f)) and 'nm' in f]
print(all_zstack_path)
all_zstack_path = sorted(all_zstack_path)

In [ ]:
viewer = napari.Viewer()
count = 0 
colors = ['red', 'green', 'blue', 'magenta', 'cyan', 'yellow', 'white']
all_zprofile = []

for zstack_path in all_zstack_path:
    print(zstack_path)
    # get folder name 
    wavelength  = os.path.basename(zstack_path)
    
    # in zstack_path find the folder starting with 'ZSeries'
    zstack_folder = [f for f in os.listdir(zstack_path) if os.path.isdir(os.path.join(zstack_path, f)) and f.startswith('ZSeries')]
    print(zstack_folder)
    if len(zstack_folder) == 0 or len(zstack_folder) > 1:
        print('No ZSeries folder found or multiple found')
        continue

    zstack_folder = zstack_folder[0]
    # now load the file edning in .tif in that folder
    zstack_file = [f for f in os.listdir(os.path.join(zstack_path, zstack_folder)) if f.endswith('.tif')]
    print(zstack_file)
    if len(zstack_file) == 0 or len(zstack_file) > 1:
        print('No .tif file found or multiple found')
        continue

    zstack_file = zstack_file[0]
    # load ome.tif file
    zstack = tf.imread(os.path.join(zstack_path, zstack_folder, zstack_file))
    zstack = zstack
    print(zstack.shape)

    zprofile = np.mean(zstack, axis=(1, 2))  # average over Y and X
    all_zprofile.append(zprofile)

    # if there is not '_stim' in name
    if '_stim' not in zstack_path:
        # show in napari
        clim = (np.min(zstack), np.max(zstack))
        viewer.add_image(zstack, name=wavelength, scale=(1.4, 1.4, 1.4), contrast_limits=clim, colormap=colors[count % len(colors)])
        count += 1

napari.run()

In [ ]:
figure = plt.figure(figsize=(8, 4))

x_steps = np.arange(0, len(all_zprofile[0])) * z_step

means = []

for (i, z) in enumerate(all_zprofile):
    # normalise to have integral of 1
    z = z / np.sum(z)

    # calculate width at half maximum
    half_max = np.max(z) / 2
    indices = np.where(z >= half_max)[0]

    # fit a gaussian to the z profile using curve_fit
    x_data = np.arange(len(z))
    initial_guess = [np.max(z), np.argmax(z), 1]

    popt, pcov = curve_fit(gaussian, x_data, z, p0=initial_guess)
    # TODO: recalculate based on central square (not whole image due to curvature/unevennenss)
    # print the means of the fitted gaussian
    means.append(popt[1] * z_step)
    print(f'Fitted mean: {popt[1] * z_step:.2f} for {all_zstack_path[i].split("/")[-1]}')
    # plot the fitted curve
    fitted_curve = gaussian(x_data, *popt)
    fwhm = 2.355 * abs(popt[2]) * 0.5
    plt.plot(x_steps, fitted_curve, '--', c=f'C{i}', label=f'Mn:{popt[1] * z_step:.1f}, fwhm: {fwhm:.1f}')
    fwhm = 2.355 * abs(popt[2]) * 0.5
    print(f'FWHM for {all_zstack_path[i].split("/")[-1]}: {fwhm:.2f} (fit)')

    plt.plot(x_steps, z, label=all_zstack_path[i].split('/')[-1], c=f'C{i}')
    # add x ticks at the means
    plt.xticks(means, labels=[f'{m:.0f}' for m in means])
    plt.axvline(means[-1], color=f'grey', alpha=0.1)
    plt.xlabel('Z (µm)')
    plt.gca().spines['top'].set_visible(False)
    plt.gca().spines['right'].set_visible(False)
    plt.gca().spines['left'].set_visible(False)
    plt.gca().yaxis.set_ticks([])
    plt.gca().yaxis.set_ticklabels([])
plt.legend(fontsize=8)
plt.savefig(os.path.join(save_path, f'{calibration}.png'), dpi=300)

In [ ]:
distances = np.diff(np.sort(np.array(means), axis=0), axis=0)
print('Distances between peaks (um): ', distances)   